Itxaso Muñoz-Aldalur

` conda env list `

` conda activate my_mayavi_env_py37 `





` sbatch *.slm `


`squeue -u operarire`



# Mag vs gamma

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
from collections import defaultdict

source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/"
plt.style.use('./mlpstyle/TEXTscience.mplstyle')


def extract_last_line(filename):
    """Same as 04.thermalization_ener.py"""
    try:
        with open(filename, 'r') as file:
            # Read all lines and return last
            lines = file.readlines()
            if lines:
                return lines[-1].strip()
    except Exception as e:
        print(f"Error reading file {filename}: {e}")
        return None
    
    
def extract_points(filename):
    """Extract points data from the configuration file."""
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
        
        # Find data lines (excluding header and comment lines)
        data_lines = [line.strip() for line in lines if line.strip() and not line.startswith('#')]
        
        # Convert lines to numpy array of points
        points_data = []
        for line in data_lines:
            values = line.split()
            
            # Ensure line has at least 8 columns (3 position + 3 spin + theta + phi)
            if len(values) >= 8:
                try:
                    # Extract position (first 3 values) and spin (next 3 values)
                    position = [float(val) for val in values[0:3]] # R(i,:)
                    spin = [float(val) for val in values[3:6]]     # Spin(i,:)
                    theta = float(values[6])                       # Theta
                    phi = float(values[7])                         # Phi
                    
                    # Store all values
                    point = position + spin + [theta, phi]
                    points_data.append(point)
                except ValueError:
                    continue
        
        # Convert to numpy array and verify we have the expected number of points
        points_array = np.array(points_data)
        
        if points_array.shape[0] != 232:
            print(f"Warning: Expected 232 points, found {points_array.shape[0]} in {filename}")
        
        return points_array
    
    except Exception as e:
        print(f"Error reading points from file {filename}: {e}")
        return None
    

def extract_parameters_from_filename(filename):
    """Extracts parameters from configuration filename."""
    # Pattern for filenames like: Conf_end_N1_4_N2_4_Tl_7.4_g_0.800.out
    pattern1 = r'Conf_end_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)\.out'
    pattern2 = r'Conf_end_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)_K_([0-9.]+)\.out'
    match1 = re.match(pattern1, os.path.basename(filename))
    match2 = re.match(pattern2, os.path.basename(filename))
    
    if match1:
        n1 = int(match1.group(1))
        n2 = int(match1.group(2))
        tl = float(match1.group(3))
        g = float(match1.group(4))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g}
    if match2:
        n1 = int(match2.group(1))
        n2 = int(match2.group(2))
        tl = float(match2.group(3))
        g = float(match2.group(4))
        K = float(match2.group(5))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g, 'K': K}
    
    return None



def create_magnetization_components_plot(source_dir, out_dir, fig_dir):
    """Plot of magnetization components vs gamma."""
    
    # Ensure figure directory exists
    os.makedirs(fig_dir, exist_ok=True)
    
    # Get list of .out files starting with "Conf_end"
    ener_files = glob.glob(os.path.join(out_dir, "Conf_end*.out"))
    
    if not ener_files:
        print(f"No files starting with 'Conf_end' found in {out_dir}")
        return
    
    # Prepare data for plot
    gamma_values = []
    m_phi_values = []
    m_z_values = []
    m_rho_values = []
    
    for file in ener_files:
        # Extract parameters from filename
        parameters = extract_parameters_from_filename(file)
        
        # Extract points data
        points = extract_points(file)
        print("Total number of spins, N:",len(points)) # Should be 232
        
        if parameters and points is not None:
            try:
                theta_values = points[:, 6] # theta values from Fortran code
                phi_values = points[:, 7]  # phi values from Fortran code
                angleR_values = np.arctan2(points[:, 1], points[:, 0]) # Angle between positions
                #print(alpha_values)
                spin_mag = np.sqrt(points[:, 3]**2 + points[:, 4]**2 + points[:, 5]**2)
                
                # Calculate cylindrical components
                spin_rho = np.sin(theta_values)*np.cos(phi_values - angleR_values)  
                total_m_rho = np.sum(spin_rho) / len(points)
  
                spin_phi = np.sin(theta_values)*np.sin(phi_values-angleR_values)
                total_m_phi = np.sum(np.abs(spin_phi)) / len(points)
                
                spin_z =  np.cos(theta_values)
                total_m_z = np.sum(np.abs(spin_z)) / len(points)
            
                # Store data
                gamma_values.append(parameters['g'])  
                m_phi_values.append(total_m_phi)
                m_z_values.append(total_m_z)
                m_rho_values.append(total_m_rho)
                
            except (ValueError, IndexError) as e:
                print(f"Error processing file {file}: {e}")
                print(f"Points shape: {points.shape if points is not None else 'None'}")
    
    # Prepare data for visualization
    if not gamma_values:
        print("No valid magnetization data found.")
        return
    
    # Sort data by gamma values
    sorted_indices = np.argsort(gamma_values)
    gamma_values = [gamma_values[i] for i in sorted_indices]
    m_phi_values = [m_phi_values[i] for i in sorted_indices]
    m_z_values = [m_z_values[i] for i in sorted_indices]
    m_rho_values = [m_rho_values[i] for i in sorted_indices]
    
    # USE SCIENTIFIC STYLE
    plt.style.use('./mlpstyle/science.mplstyle')
    
    # Create the plot
    plt.figure()  
    #plt.figure(figsize=(4.5, 3.5))

    # Plot each component
    plt.plot(gamma_values, m_phi_values, 'ro-', label=r'$m_\varphi$')
    plt.plot(gamma_values, m_z_values, 'go-', label=r'$m_z$')
    plt.plot(gamma_values, m_rho_values, 'bo-', label=r'$m_\rho$')
    plt.xlabel(r'$\gamma$')
    plt.ylabel(r'$M/M_s$')
    
    ax = plt.gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))                # Integer x ticks
    
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'magnetization_polar_components.pdf'))
    plt.close()
    
    print(f"Magnetization components plot saved in {fig_dir}")
    

def main():
    # Directory containing the .out files
    source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/"
    out_dir = os.path.join(source_dir, "out_plots")
    fig_dir = os.path.join(source_dir, "BigPlots")
    
    # Create magnetization components plot
    create_magnetization_components_plot(source_dir, out_dir, fig_dir)

if __name__ == "__main__":
    main()

Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Total number of spins, N: 240
Magnetization components plot saved in /Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/BigPlots


## AA, AB

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MaxNLocator, FormatStrFormatter
from matplotlib.ticker import MultipleLocator
from collections import defaultdict

source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/"
plt.style.use('./mlpstyle/TEXTscience.mplstyle')


def extract_last_line(filename):
    """Same as 04.thermalization_ener.py"""
    try:
        with open(filename, 'r') as file:
            # Read all lines and return last
            lines = file.readlines()
            if lines:
                return lines[-1].strip()
    except Exception as e:
        print(f"Error reading file {filename}: {e}")
        return None
    
    
def extract_points(filename):
    """Extract points data from the configuration file."""
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
        
        # Find data lines (excluding header and comment lines)
        data_lines = [line.strip() for line in lines if line.strip() and not line.startswith('#')]
        
        # Convert lines to numpy array of points
        points_data = []
        for line in data_lines:
            values = line.split()
            
            # Ensure line has at least 8 columns (3 position + 3 spin + theta + phi)
            if len(values) >= 8:
                try:
                    # Extract position (first 3 values) and spin (next 3 values)
                    position = [float(val) for val in values[0:3]] # R(i,:)
                    spin = [float(val) for val in values[3:6]]     # Spin(i,:)
                    theta = float(values[6])                       # Theta
                    phi = float(values[7])                         # Phi
                    
                    # Store all values
                    point = position + spin + [theta, phi]
                    points_data.append(point)
                except ValueError:
                    continue
        
        # Convert to numpy array and verify we have the expected number of points
        points_array = np.array(points_data)
        
        if points_array.shape[0] != 232:
            print(f"Warning: Expected 232 points, found {points_array.shape[0]} in {filename}")
        
        return points_array
    
    except Exception as e:
        print(f"Error reading points from file {filename}: {e}")
        return None
    

def extract_parameters_from_filename(filename):
    """Extracts parameters from configuration filename using a more flexible pattern."""    # IMPLEMENT K!
    base_name = os.path.basename(filename)
    
    # Extract N1
    n1_match = re.search(r'N1_(\d+)', base_name)
    n1 = int(n1_match.group(1)) if n1_match else 0
    
    # Extract N2 (which might be missing or empty)
    n2_match = re.search(r'N2_(\d+)', base_name)
    n2 = int(n2_match.group(1)) if n2_match else 0
    
    # Extract Tl
    tl_match = re.search(r'Tl_([0-9.]+)', base_name)
    tl = float(tl_match.group(1)) if tl_match else 0.0
    
    # Extract g and handle trailing dots
    g_match = re.search(r'g_([0-9.]+)', base_name)
    if g_match:
        g_value = g_match.group(1).rstrip('.')  # Remove any trailing dots
        try:
            g = float(g_value)
        except ValueError as e:
            print(f"Error converting g value '{g_value}' in file {base_name}: {e}")
            g = 0.0
    else:
        g = 0.0
    
    if n1_match and tl_match and g_match:  # Ensure we found the essential parameters
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g}
    else:
        print(f"Warning: Could not extract all parameters from {base_name}")
        return None



def create_magnetization_components_plot(source_dir, out_dir, fig_dir):
    """Plot of magnetization components vs gamma."""
    
    # Ensure figure directory exists
    os.makedirs(fig_dir, exist_ok=True)
    
    # Get list of .out files starting with "Conf_end"
    ener_files = glob.glob(os.path.join(out_dir, "Conf_end*.out"))
    
    if not ener_files:
        print(f"No files starting with 'Conf_end' found in {out_dir}")
        return
    
    # Prepare data for plot
    gamma_values = []
    m_phi_values = []
    m_z_values = []
    m_rho_values = []
    
    for file in ener_files:
        try:
            # Extract parameters from filename
            parameters = extract_parameters_from_filename(file)
            
            if not parameters:
                print(f"Skipping file due to invalid parameters: {file}")
                continue
                
            # Extract points data
            points = extract_points(file)
            
            if points is None or len(points) == 0:
                print(f"Skipping file due to invalid points data: {file}")
                continue
                
            print("Total number of spins, N:", len(points))
            
            theta_values = points[:, 6] # theta values from Fortran code
            phi_values = points[:, 7]  # phi values from Fortran code
            angleR_values = np.arctan2(points[:, 1], points[:, 0]) # Angle between positions
            
            # Calculate cylindrical components
            spin_rho = np.sin(theta_values)*np.cos(phi_values - angleR_values)  
            total_m_rho = np.sum(spin_rho) / len(points)

            spin_phi = np.sin(theta_values)*np.sin(phi_values-angleR_values)
            total_m_phi = np.sum(np.abs(spin_phi)) / len(points)
            
            spin_z = np.cos(theta_values)
            total_m_z = np.sum(np.abs(spin_z)) / len(points)
        
            # Store data
            gamma_values.append(parameters['g'])  
            m_phi_values.append(total_m_phi)
            m_z_values.append(total_m_z)
            m_rho_values.append(total_m_rho)
                
        except Exception as e:
            print(f"Error processing file {file}: {e}")
            if 'points' in locals() and points is not None:
                print(f"Points shape: {points.shape}")
    
    # Prepare data for visualization
    if not gamma_values:
        print("No valid magnetization data found.")
        return
    
    # Sort data by gamma values
    sorted_indices = np.argsort(gamma_values)
    gamma_values = [gamma_values[i] for i in sorted_indices]
    m_phi_values = [m_phi_values[i] for i in sorted_indices]
    m_z_values = [m_z_values[i] for i in sorted_indices]
    m_rho_values = [m_rho_values[i] for i in sorted_indices]
    
    # USE SCIENTIFIC STYLE
    plt.style.use('./mlpstyle/science.mplstyle')
    
    # Create the plot
    plt.figure()  
    #plt.figure(figsize=(4.5, 3.5))
    
    #plt.title(r'J=0, g=1  J=1, g=1  J=1, g=1')

    # Plot each component
    plt.plot(gamma_values, m_phi_values, 'ro-', label=r'$m_\varphi$')
    plt.plot(gamma_values, m_z_values, 'go-', label=r'$m_z$')
    plt.plot(gamma_values, m_rho_values, 'bo-', label=r'$m_\rho$')
    plt.xlabel(r'$\gamma$')
    plt.ylabel(r'$M/M_s$')
    #plt.legend()
    
    ax = plt.gca()
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'magnetization_polar_components.pdf'))
    plt.close()
    
    

def main():
    # Directory containing the .out files
    source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/"
    out_dir = os.path.join(source_dir, "out_plots")
    fig_dir = os.path.join(source_dir, "BigPlots")
    
    # Create magnetization components plot
    create_magnetization_components_plot(source_dir, out_dir, fig_dir)

if __name__ == "__main__":
    main()

Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total numb

# Ani

## ✅ 4.4. M vs K, but for Mrho as sum of abs val of projections

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
from collections import defaultdict

source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/"
plt.style.use('./mlpstyle/TEXTscience.mplstyle')


def extract_last_line(filename):
    """Same as 04.thermalization_ener.py"""
    try:
        with open(filename, 'r') as file:
            # Read all lines and return last
            lines = file.readlines()
            if lines:
                return lines[-1].strip()
    except Exception as e:
        print(f"Error reading file {filename}: {e}")
        return None
    
    
def extract_points(filename):
    """Extract points data from the configuration file."""
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
        
        # Find data lines (excluding header and comment lines)
        data_lines = [line.strip() for line in lines if line.strip() and not line.startswith('#')]
        
        # Convert lines to numpy array of points
        points_data = []
        for line in data_lines:
            values = line.split()
            
            # Ensure line has at least 8 columns (3 position + 3 spin + theta + phi)
            if len(values) >= 8:
                try:
                    # Extract position (first 3 values) and spin (next 3 values)
                    position = [float(val) for val in values[0:3]] # R(i,:)
                    spin = [float(val) for val in values[3:6]]     # Spin(i,:)
                    theta = float(values[6])                       # Theta
                    phi = float(values[7])                         # Phi
                    
                    # Store all values
                    point = position + spin + [theta, phi]
                    points_data.append(point)
                except ValueError:
                    continue
        
        # Convert to numpy array and verify we have the expected number of points
        points_array = np.array(points_data)
        
        if points_array.shape[0] != 232:
            print(f"Warning: Expected 232 points, found {points_array.shape[0]} in {filename}")
        
        return points_array
    
    except Exception as e:
        print(f"Error reading points from file {filename}: {e}")
        return None
    

def extract_parameters_from_filename(filename):
    """Extracts parameters from configuration filename."""
    # Pattern for filenames like: Conf_end_N1_4_N2_4_Tl_7.4_g_0.800.out
    pattern1 = r'Conf_end_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)\.out'
    pattern2 = r'Conf_end_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)_K_([0-9.]+)\.out'
    match1 = re.match(pattern1, os.path.basename(filename))
    match2 = re.match(pattern2, os.path.basename(filename))
    
    if match1:
        n1 = int(match1.group(1))
        n2 = int(match1.group(2))
        tl = float(match1.group(3))
        g = float(match1.group(4))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g}
    if match2:
        n1 = int(match2.group(1))
        n2 = int(match2.group(2))
        tl = float(match2.group(3))
        g = float(match2.group(4))
        K = float(match2.group(5))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g, 'K': K}
    
    return None



def create_magnetization_components_plot(source_dir, out_dir, fig_dir):
    """Plot of magnetization components vs gamma."""
    
    # Ensure figure directory exists
    os.makedirs(fig_dir, exist_ok=True)
    
    # Get list of .out files starting with "Conf_end"
    ener_files = glob.glob(os.path.join(out_dir, "Conf_end*.out"))
    
    if not ener_files:
        print(f"No files starting with 'Conf_end' found in {out_dir}")
        return
    
    # Prepare data for plot
    K_values = []
    m_phi_values = []
    m_z_values = []
    m_rho_values = []
    
    for file in ener_files:
        # Extract parameters from filename
        parameters = extract_parameters_from_filename(file)
        
        # Extract points data
        points = extract_points(file)
        print("Total number of spins, N:",len(points)) # Should be 232
        
        if parameters and points is not None:
            try:
                theta_values = points[:, 6] # theta values from Fortran code
                phi_values = points[:, 7]  # phi values from Fortran code
                angleR_values = np.arctan2(points[:, 1], points[:, 0]) # Angle between positions
                #print(alpha_values)
                spin_mag = np.sqrt(points[:, 3]**2 + points[:, 4]**2 + points[:, 5]**2)
                
                # Calculate cylindrical components
                spin_rho = np.abs(np.sin(theta_values)*np.cos(phi_values - angleR_values))  
                total_m_rho = np.sum(spin_rho) / len(points)
  
                spin_phi = np.sin(theta_values)*np.sin(phi_values-angleR_values)
                total_m_phi = np.sum(np.abs(spin_phi)) / len(points)
                
                spin_z =  np.cos(theta_values)
                total_m_z = np.sum(np.abs(spin_z)) / len(points)
            
                # Store data
                K_values.append(parameters['K'])  
                m_phi_values.append(total_m_phi)
                m_z_values.append(total_m_z)
                m_rho_values.append(total_m_rho)
                
            except (ValueError, IndexError) as e:
                print(f"Error processing file {file}: {e}")
                print(f"Points shape: {points.shape if points is not None else 'None'}")
    
    # Prepare data for visualization
    if not K_values:
        print("No valid magnetization data found.")
        return
    
    # Sort data by gamma values
    sorted_indices = np.argsort(K_values)
    K_values = [K_values[i] for i in sorted_indices]
    m_phi_values = [m_phi_values[i] for i in sorted_indices]
    m_z_values = [m_z_values[i] for i in sorted_indices]
    m_rho_values = [m_rho_values[i] for i in sorted_indices]
    
    
    # Create the plot
    plt.figure()  

    # Plot each component
    plt.plot(K_values, m_phi_values, 'ro-', label=r'$m_\varphi$')
    plt.plot(K_values, m_z_values, 'go-', label=r'$m_z$')
    plt.plot(K_values, m_rho_values, 'bo-', label=r'$|m_\rho|$')
    plt.xlabel(r'$K$ ')
    plt.ylabel(r'$M/M_s$')
    #plt.title('Magnetization polar components vs K (J=0)')
    #plt.legend()
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    
    ax = plt.gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))                # Integer x ticks, every 5 vals
    ax.xaxis.set_major_locator(MultipleLocator(2))
    
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'm_vs_K.pdf'))
    plt.close()
    

    
    print(f"Magnetization components plot saved in {fig_dir}")
    

def main():
    # Directory containing the .out files
    source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/"
    out_dir = os.path.join(source_dir, "out_plots")
    fig_dir = os.path.join(source_dir, "BigPlots")
    
    # Create magnetization components plot
    create_magnetization_components_plot(source_dir, out_dir, fig_dir)

if __name__ == "__main__":
    main()


Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total number of spins, N: 232
Total numb

# Energies: E_dip, E_ani, E_ex in function of K ✅

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from matplotlib.ticker import AutoMinorLocator
from collections import defaultdict

source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/out"
plt.style.use('./mlpstyle/TEXTscience.mplstyle')


def extract_parameters_from_filename(filename):
    """Extracts parameters from configuration filename."""
    # Pattern for filenames like: Conf_end_N1_4_N2_4_Tl_7.4_g_0.800.out
    pattern1 = r'Ener_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)\.out'
    pattern2 = r'Ener_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)_K_([0-9.]+)\.out'
    match1 = re.match(pattern1, os.path.basename(filename))
    match2 = re.match(pattern2, os.path.basename(filename))
    
    if match1:
        n1 = int(match1.group(1))
        n2 = int(match1.group(2))
        tl = float(match1.group(3))
        g = float(match1.group(4))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g}
    if match2:
        n1 = int(match2.group(1))
        n2 = int(match2.group(2))
        tl = float(match2.group(3))
        g = float(match2.group(4))
        K = float(match2.group(5))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g, 'K': K}
    
    return None


    
def extract_parameters_from_thermalization(filename):
    """Extracts parameters from the thermalization file """
    
    data = {'filename': [],'energy_first_column': [],
            'values': {'temperature': [],'E_tot': [],'E_dip': [],'E_ex': [],'E_ani': [],
                   'mag_x': [],'mag_y': [],'mag_z': [],'mag_mod': [],
                   'specific_heat': [],'susceptibility': []}}
    
    try:
        for line in filename:
            # Split the line by ':' to separate filename from values
            parts = line.split(':')
    
            # Extract filename
            filename_full = parts[0].strip()
            data['filename'].append(filename_full)
            
            # Extract numerical values
            values = parts[1].strip().split()
            
            # Store the first column separately if needed
            data['energy_first_column'].append(float(values[0]))
            
            # Store other columns
            for i, key in enumerate(data['values'].keys(), start=0):
                data['values'][key].append(float(values[i]))
    
    except Exception as e:
        print(f"An error occurred while processing the data: {e}")
        return None
    
    return data


def create_energy_vs_K_plots(therm_file, fig_dir):
    """ Create plots for E_dip and E_exchange vs K from thermalization data."""
    
    # Ensure figure directory exists
    os.makedirs(fig_dir, exist_ok=True)
    
    # Read the thermalization file
    try:
        with open(therm_file, 'r') as file:
            lines = file.readlines()
    except Exception as e:
        print(f"Error reading file {therm_file}: {e}")
        return
    
    # Extract data
    data = extract_parameters_from_thermalization(lines)
    if not data:
        print("Failed to extract data from thermalization file")
        return
    
    # Prepare data for plot
    K_values = []
    e_dip_values = []
    e_ani_values = []
    e_ex_values = []
    
    
    for i, filename in enumerate(data['filename']):
        try:
            # Extract parameters from filename
            parameters = extract_parameters_from_filename(filename)
            
            if not parameters:
                print(f"Skipping file due to invalid parameters: {filename}")
                continue
                
            # Store data
            K_values.append(parameters['K']) 
            e_dip_values.append(data['values']['E_dip'][i])
            e_ani_values.append(data['values']['E_ani'][i])
            e_ex_values.append(data['values']['E_ex'][i])
                
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
    
    # Prepare data for visualization
    if not K_values:
        print("No valid energy data found.")
        return
    
    # Sort data by K values
    sorted_indices = np.argsort(K_values)
    K_values = [K_values[i] for i in sorted_indices]
    e_dip_values = [e_dip_values[i] for i in sorted_indices]
    e_ani_values = [e_ani_values[i]*K_values[i] for i in sorted_indices]
    e_ex_values = [e_ex_values[i] for i in sorted_indices]
    
    
    
    # Create E_dip plot
    plt.figure()  
    
    
# Plot dipolar energy
    plt.plot(K_values, e_dip_values, 'bo-', label=r'$E_{dip}$')  
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
# Plot anisotropy energy
    plt.plot(K_values, e_ani_values, 'ro-', label=r'$E_{ani}$')
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
# Plot exchange energy
    plt.plot(K_values, e_ex_values, 'go-', label=r'$E_{ex}$')
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    
    # Legend, axis and title
    plt.xlabel(r'$K$')
    plt.ylabel(r'$E$')
    #plt.title('Energies vs K (J=1,g=1)')
    
    #plt.legend()
    
    # Set the axis limits
    #plt.xlim(0, 1000) 
    #plt.ylim(-6, 0.1)   
    
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'Edip_Eani_Eex_vs_K.pdf'))
    plt.close()
    
    print(f"Exchange energy plot saved in {fig_dir}")
    

def main():
    # Directory containing the thermalization file and where to save figures
    source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/out"
    therm_file = os.path.join(source_dir, "Thermalization_Ener.out") 
    fig_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/BigPlots"
    
    # Create energy vs K plots
    create_energy_vs_K_plots(therm_file, fig_dir)

if __name__ == "__main__":
    main()

Exchange energy plot saved in /Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/BigPlots


# Energies + E TOT in function of K ✅

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
from collections import defaultdict

source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/out"
plt.style.use('./mlpstyle/TEXTscience.mplstyle')


def extract_parameters_from_filename(filename):
    """Extracts parameters from configuration filename."""
    # Pattern for filenames like: Conf_end_N1_4_N2_4_Tl_7.4_g_0.800.out
    pattern1 = r'Ener_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)\.out'
    pattern2 = r'Ener_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)_K_([0-9.]+)\.out'
    match1 = re.match(pattern1, os.path.basename(filename))
    match2 = re.match(pattern2, os.path.basename(filename))
    
    if match1:
        n1 = int(match1.group(1))
        n2 = int(match1.group(2))
        tl = float(match1.group(3))
        g = float(match1.group(4))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g}
    if match2:
        n1 = int(match2.group(1))
        n2 = int(match2.group(2))
        tl = float(match2.group(3))
        g = float(match2.group(4))
        K = float(match2.group(5))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g, 'K': K}
    
    return None


    
def extract_parameters_from_thermalization(filename):
    """Extracts parameters from the thermalization file """
    
    data = {'filename': [],'energy_first_column': [],
            'values': {'temperature': [],'E_tot': [],'E_dip': [],'E_ex': [],'E_ani': [],
                   'mag_x': [],'mag_y': [],'mag_z': [],'mag_mod': [],
                   'specific_heat': [],'susceptibility': []}}
    
    try:
        for line in filename:
            # Split the line by ':' to separate filename from values
            parts = line.split(':')
    
            # Extract filename
            filename_full = parts[0].strip()
            data['filename'].append(filename_full)
            
            # Extract numerical values
            values = parts[1].strip().split()
            
            # Store the first column separately if needed
            data['energy_first_column'].append(float(values[0]))
            
            # Store other columns
            for i, key in enumerate(data['values'].keys(), start=0):
                data['values'][key].append(float(values[i]))
    
    except Exception as e:
        print(f"An error occurred while processing the data: {e}")
        return None
    
    return data


def create_energy_vs_K_plots(therm_file, fig_dir):
    """ Create plots for E_dip and E_exchange vs K from thermalization data."""
    
    # Ensure figure directory exists
    os.makedirs(fig_dir, exist_ok=True)
    
    # Read the thermalization file
    try:
        with open(therm_file, 'r') as file:
            lines = file.readlines()
    except Exception as e:
        print(f"Error reading file {therm_file}: {e}")
        return
    
    # Extract data
    data = extract_parameters_from_thermalization(lines)
    if not data:
        print("Failed to extract data from thermalization file")
        return
    
    # Prepare data for plot
    K_values = []
    e_tot_values = []
    e_dip_values = []
    e_ex_values = []
    e_ani_values = []
    e_dipex_values = []
    
    for i, filename in enumerate(data['filename']):
        try:
            # Extract parameters from filename
            parameters = extract_parameters_from_filename(filename)
            
            if not parameters:
                print(f"Skipping file due to invalid parameters: {filename}")
                continue
                
            # Store data
            K_values.append(parameters['K']) 
            e_ani_values.append(data['values']['E_ani'][i])
            e_dip_values.append(data['values']['E_dip'][i])
            e_ex_values.append(data['values']['E_ex'][i])
                
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
    
    # Prepare data for visualization
    if not K_values:
        print("No valid energy data found.")
        return
    
    # Sort data by K values
    sorted_indices = np.argsort(K_values)
    K_values = [K_values[i] for i in sorted_indices]
    e_dipex_values = [e_dip_values[i]+e_ex_values[i] for i in sorted_indices]
    e_tot_values = [e_dip_values[i]+e_ex_values[i]+e_ani_values[i]*K_values[i] for i in sorted_indices]
    e_dip_values = [e_dip_values[i] for i in sorted_indices]
    e_ani_values = [e_ani_values[i]*K_values[i] for i in sorted_indices]
    e_ex_values = [e_ex_values[i] for i in sorted_indices]
    
    
    
    # Create E_dip plot
    plt.figure()  
    
    
# Plot total energy
    plt.plot(K_values, e_tot_values,  label=r'$E_{tot}$')  
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
# Plot dipolar energy
    plt.plot(K_values, e_dip_values,  label=r'$E_{dip}$')  
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
# Plot anisotropy energy
    plt.plot(K_values, e_ani_values,  label=r'$E_{ani}$')
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
# Plot exchange energy
    plt.plot(K_values, e_ex_values,  label=r'$E_{ex}$')
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    # Legend, axis and title
    plt.xlabel(r'$K$')
    plt.ylabel(r'$E$')
    #plt.title('Total and Dipolar+Exchange energies vs K (J=1,g=1)')
    
    #plt.legend()
    
    plt.ylim(-15, 0.5) 
    
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))                # Integer x ticks, every 5 vals
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))                # Integer y ticks, every 2 vals
    ax.yaxis.set_major_locator(MultipleLocator(3))
     
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'Etot_Edip+ex_vs_K.pdf'))
    plt.close()
    
    print(f"Exchange energy plot saved in {fig_dir}")
    

def main():
    # Directory containing the thermalization file and where to save figures
    source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/out"
    therm_file = os.path.join(source_dir, "Thermalization_Ener.out") 
    fig_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/BigPlots"
    
    # Create energy vs K plots
    create_energy_vs_K_plots(therm_file, fig_dir)

if __name__ == "__main__":
    main()

Error reading file /Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/out/Thermalization_Ener.out: [Errno 2] No such file or directory: '/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/out/Thermalization_Ener.out'


# Energies + E TOT in function of gamma

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
from collections import defaultdict

source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/out"
plt.style.use('./mlpstyle/science.mplstyle')


def extract_parameters_from_filename(filename):
    """Extracts parameters from configuration filename."""
    # Pattern for filenames like: Conf_end_N1_4_N2_4_Tl_7.4_g_0.800.out
    pattern1 = r'Ener_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)\.out'
    pattern2 = r'Ener_N1_(\d+)_N2_(\d+)_Tl_([0-9.]+)_g_([0-9.]+)_K_([0-9.]+)\.out'
    match1 = re.match(pattern1, os.path.basename(filename))
    match2 = re.match(pattern2, os.path.basename(filename))
    
    if match1:
        n1 = int(match1.group(1))
        n2 = int(match1.group(2))
        tl = float(match1.group(3))
        g = float(match1.group(4))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g}
    if match2:
        n1 = int(match2.group(1))
        n2 = int(match2.group(2))
        tl = float(match2.group(3))
        g = float(match2.group(4))
        K = float(match2.group(5))
        return {'N1': n1, 'N2': n2, 'Tl': tl, 'g': g, 'K': K}
    
    return None


    
def extract_parameters_from_thermalization(filename):
    """Extracts parameters from the thermalization file """
    
    data = {'filename': [],'energy_first_column': [],
            'values': {'temperature': [],'E_tot': [],'E_dip': [],'E_ex': [],'E_ani': [],
                   'mag_x': [],'mag_y': [],'mag_z': [],'mag_mod': [],
                   'specific_heat': [],'susceptibility': []}}
    
    try:
        for line in filename:
            # Split the line by ':' to separate filename from values
            parts = line.split(':')
    
            # Extract filename
            filename_full = parts[0].strip()
            data['filename'].append(filename_full)
            
            # Extract numerical values
            values = parts[1].strip().split()
            
            # Store the first column separately if needed
            data['energy_first_column'].append(float(values[0]))
            
            # Store other columns
            for i, key in enumerate(data['values'].keys(), start=0):
                data['values'][key].append(float(values[i]))
    
    except Exception as e:
        print(f"An error occurred while processing the data: {e}")
        return None
    
    return data


def create_energy_vs_gamma_plots(therm_file, fig_dir):
    """ Create plots for E_tot, E_ani, E_dip, E_exchange vs g from thermalization data."""
    
    # Ensure figure directory exists
    os.makedirs(fig_dir, exist_ok=True)
    
    # Read the thermalization file
    try:
        with open(therm_file, 'r') as file:
            lines = file.readlines()
    except Exception as e:
        print(f"Error reading file {therm_file}: {e}")
        return
    
    # Extract data
    data = extract_parameters_from_thermalization(lines)
    if not data:
        print("Failed to extract data from thermalization file")
        return
    
    # Prepare data for plot
    K_values = []
    gamma_values = []
    e_tot_values = []
    e_dip_values = []
    e_ex_values = []
    e_ani_values = []
    e_dipex_values = []
    
    for i, filename in enumerate(data['filename']):
        try:
            # Extract parameters from filename
            parameters = extract_parameters_from_filename(filename)
            
            if not parameters:
                print(f"Skipping file due to invalid parameters: {filename}")
                continue
                
            # Store data
            gamma_values.append(parameters['g']) 
            K_values.append(parameters['K']) 
            e_ani_values.append(data['values']['E_ani'][i])
            e_dip_values.append(data['values']['E_dip'][i])
            e_ex_values.append(data['values']['E_ex'][i])
                
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
    
    # Prepare data for visualization
    if not gamma_values:
        print("No valid energy data found.")
        return
    
    # Sort data by g values
    sorted_indices = np.argsort(gamma_values)
    K_values = [K_values[i] for i in sorted_indices]
    gamma_values = [gamma_values[i] for i in sorted_indices]
    e_dipex_values = [e_dip_values[i]+e_ex_values[i] for i in sorted_indices]
    e_tot_values = [e_dip_values[i]+e_ex_values[i]+e_ani_values[i]*K_values[i] for i in sorted_indices]
    e_dip_values = [e_dip_values[i] for i in sorted_indices]
    e_ani_values = [e_ani_values[i]*K_values[i] for i in sorted_indices]
    e_ex_values = [e_ex_values[i] for i in sorted_indices]
    
    
    
    # Create E_dip plot
    plt.figure()  
    
    
# Plot total energy
    plt.plot(gamma_values, e_tot_values,  label=r'$E_{tot}$')  
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
# Plot dipolar energy
    plt.plot(gamma_values, e_dip_values,  label=r'$E_{dip}$')  
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
# Plot anisotropy energy
    plt.plot(gamma_values, e_ani_values,  label=r'$E_{ani}$')
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
# Plot exchange energy
    plt.plot(gamma_values, e_ex_values,  label=r'$E_{ex}$')
    # Add minor ticks
    ax = plt.gca()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    # Legend, axis and title
    plt.xlabel(r'$\gamma$')
    plt.ylabel(r'$E$')
    plt.title('K=2')
    
    plt.legend()
    
    #plt.ylim(-15, 0.5) 
    
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))                # Integer x ticks, every 5 vals
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))                # Integer y ticks, every 2 vals
    ax.yaxis.set_major_locator(MultipleLocator(3))
     
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'Etot_vs_g.pdf'))
    plt.close()
    
    print(f"Exchange energy plot saved in {fig_dir}")
    

def main():
    # Directory containing the thermalization file and where to save figures
    source_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/out"
    therm_file = os.path.join(source_dir, "Thermalization_Ener.out") 
    fig_dir = "/Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/BigPlots"
    
    # Create energy vs K plots
    create_energy_vs_gamma_plots(therm_file, fig_dir)

if __name__ == "__main__":
    main()

Exchange energy plot saved in /Users/itxaso/codiTFG/ChiralStructuresProject/ChiralStructures25/BigPlots
